# Loading Data

In [ ]:
import os

for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os

for dirname, _, filenames in os.walk('../working/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
import sys
import time
import re
import string
import pickle
import copy

import numpy as np
from numpy.linalg import inv

import torch
# import pandas as pd
from collections import Counter

from scipy.stats import zscore
import scipy.io as sio

In [ ]:
class HP_dataset_denoised(torch.utils.data.Dataset):
	def __init__(self,args):
		self.args = args

		self.all_texts, self.all_megs = self.load_words_and_megs()

		self.uniq_words = self.get_uniq_words()
		self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
		self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

		self.texts, self.megs = self.load_data()

	def load_words_and_megs(self, do_zscore = True):
		if self.args.chapter==1:
			mat = sio.loadmat(self.args.base_data_path+"A_HP_notDetrended_25ms.mat")
			words = mat['labels']
			all_texts=[w[0][0].replace("@","").replace("\\","").replace("+","").replace("^","").strip() for w in words][:5176]
			
			all_megs=np.load(self.args.base_data_path+"denoised_HP.npy")
			all_megs=np.swapaxes(all_megs,1,2)
		elif self.args.chapter==2:
			words = np.load(self.args.base_data_path+"words_chapter_2.npy")
			all_texts=[w.replace("@","").replace("\\","").replace("+","").replace("^","").strip() for w in words]
			
			all_megs=np.load(self.args.base_data_path+"denoised_HP_chapter2.npy")
			all_megs=np.swapaxes(all_megs,1,2)

		if do_zscore:
			all_megs = zscore(all_megs)

		return all_texts, all_megs

	def load_data(self):
		texts_to_load=list()
		megs_to_load=list()
		
		megs=np.mean(self.all_megs[:,:,12:17],axis=2)
		for location in range(self.args.sequence_length,len(self.all_texts)):
			texts=self.all_texts[location-self.args.sequence_length+1:location+1]
			texts_to_load.append([self.word_to_index[w] for w in texts])
			megs_to_load.append(megs[location])
		
		texts_to_load=np.vstack(texts_to_load)
		megs_to_load=np.vstack(megs_to_load)
		assert len(texts_to_load)==len(megs_to_load)
		return texts_to_load, megs_to_load

	def get_uniq_words(self):
		words=set()
		for word in self.all_texts:
			words.add(word)
		return list(words)

	def __len__(self):
		return len(self.texts) 

	def __getitem__(self, index):
		# return word, meg
		return(
			torch.tensor(self.texts[index]),
			torch.tensor(self.megs[index+self.args.meg_offset])
		)

# Utils

In [ ]:
import matplotlib.pyplot as plt

from scipy.special import softmax
from scipy.stats import pearsonr
from scipy.stats import chi2

from sklearn.model_selection import KFold

import statsmodels.stats.multitest as smm

import torch

import mne
from mne.io import read_raw_ctf

# from utils.ridge.ridge import bootstrap_ridge

In [ ]:


def correlation_loss_wordwise(x,y):
	return correlation_loss(x,y,'wordwise')

def correlation_loss_channelwise(x,y):
	return correlation_loss(x,y,'channelwise')

def correlation_loss(x,y,mode):
	corr_list=correlation(x,y,mode)
	return -torch.mean(corr_list) # because we want to minimize loss/maximize correlation

def correlation(x,y,mode="channelwise"):
	valide_modes={'channelwise','wordwise'}
	if mode not in valide_modes:
		raise ValueError("Unknown Correlation Loss Function.")

	if isinstance(x, np.ndarray) and isinstance(y, np.ndarray):
		x=torch.tensor(x)
		y=torch.tensor(y)
	
	# if channelwise correlation
	xx=x.reshape(-1,x.shape[-1])
	yy=y.reshape(-1,y.shape[-1])
	# if wordwise correlation
	if mode=="wordwise":
		xx=xx.transpose(0, 1)
		yy=yy.transpose(0, 1)
		
	# print(xx.shape,yy.shape)
	# print(torch.mean(xx,axis=0).shape,torch.mean(yy,axis=0).shape)
	vx = xx - torch.mean(xx,axis=0)
	vy = yy - torch.mean(yy,axis=0)
	# print(torch.sum(vx * vy, dim=0).shape)
	cor_list=torch.sum(vx * vy, dim=0) * torch.rsqrt(torch.sum(vx ** 2,dim=0)) * torch.rsqrt(torch.sum(vy ** 2,dim=0))
	# print(cor_list.shape, cor_list)
	return cor_list

def compute_brain_surprisal_according_to_polarity(brain_data,neg_sig_channels=None,pos_sig_channels=None):
	# brain_data: N_data_point x N_channel
	if neg_sig_channels and pos_sig_channels:
		temp1=-brain_data[:,neg_sig_channels]
		temp2=brain_data[:,pos_sig_channels]
		temp=np.hstack([temp1,temp2])
		return np.mean(temp,axis=1)
	else:
		return np.mean(brain_data,axis=1)

def load_tokenizer_and_model_from_transformers(name):
	if name=="GPT-J":
		from transformers import AutoTokenizer, GPTJForCausalLM
		tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B",truncation_side="left",padding_side="left")
		model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B",output_hidden_states=True)
	elif name=="demo_GPT":
		from transformers import AutoTokenizer, GPTJForCausalLM
		tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-gptj",truncation_side="left",padding_side="left")
		model = GPTJForCausalLM.from_pretrained("hf-internal-testing/tiny-random-gptj",output_hidden_states=True)	
	elif name=="GPT2":
		from transformers import AutoTokenizer, GPT2LMHeadModel
		tokenizer = AutoTokenizer.from_pretrained("gpt2",truncation_side="left",padding_side="left")
		model = GPT2LMHeadModel.from_pretrained("gpt2",output_hidden_states=True)
	elif name=="GPT2-xl":
		from transformers import AutoTokenizer, GPT2LMHeadModel
		tokenizer = AutoTokenizer.from_pretrained("gpt2-xl",truncation_side="left",padding_side="left")
		model = GPT2LMHeadModel.from_pretrained("gpt2-xl",output_hidden_states=True)
	elif name=="GPT-Neo":
		from transformers import AutoTokenizer, GPTNeoForCausalLM
		tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B",truncation_side="left",padding_side="left")
		model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B",output_hidden_states=True)	
	elif name=="Llama-2":
		from transformers import AutoTokenizer, LlamaForCausalLM
		tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf",truncation_side="left",padding_side="left")
		model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",output_hidden_states=True)	
	elif name=="GPT-J_bare":
		from transformers import AutoTokenizer, GPTJModel
		tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B",truncation_side="left",padding_side="left")
		model = GPTJModel.from_pretrained("EleutherAI/gpt-j-6B")
	elif name=="demo_GPT_bare":
		from transformers import AutoTokenizer, GPTJModel
		tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-gptj",truncation_side="left",padding_side="left")
		model = GPTJModel.from_pretrained("hf-internal-testing/tiny-random-gptj")
	elif name=="GPT2_bare":
		from transformers import GPT2Tokenizer, GPT2Model
		tokenizer = GPT2Tokenizer.from_pretrained("gpt2",truncation_side="left",padding_side="left")
		model = GPT2Model.from_pretrained("gpt2")
	elif name=="GPT2-xl_bare":
		from transformers import GPT2Tokenizer, GPT2Model
		tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl",truncation_side="left",padding_side="left")
		model = GPT2Model.from_pretrained("gpt2-xl")
	elif name=="GPT-Neo_bare":
		from transformers import AutoTokenizer, GPTNeoModel
		tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B",truncation_side="left",padding_side="left")
		model = GPTNeoModel.from_pretrained("EleutherAI/gpt-neo-2.7B")
	elif name=="Llama-2_bare":
		from transformers import AutoTokenizer, LlamaModel
		tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf",truncation_side="left",padding_side="left")
		model = LlamaModel.from_pretrained("meta-llama/Llama-2-7b-hf")
	else:
		raise ValueError("Unknown model name.")
	return tokenizer, model

def find_indices_of_last_word(tokenizer,last_word,encodings):
	for word_end in range(len(encodings)-1,-1,-1):
		for word_start in range(word_end,-1,-1):
			reconstructed="".join([tokenizer.decode(i) for i in encodings[word_start:word_end+1]]).strip()
			if reconstructed==last_word:
				return (word_start, word_end)
	return (len(encodings)-1,len(encodings)-1)

def find_indices_of_last_word_in_batch(tokenizer,batch_last_words,batch_encodings):
	assert len(batch_last_words)==len(batch_encodings)
	return [find_indices_of_last_word(tokenizer,last_word,encodings) for last_word,encodings in zip(batch_last_words,batch_encodings)]

def compute_surprisal(all_last_words,all_logits):    
	assert len(all_logits)==len(all_last_words)
	all_surprisal=list()
	for token_ids, logits in zip(all_last_words,all_logits):
		assert len(token_ids)==len(logits)
		surprisal=0
		for token_id,logit in zip(token_ids,logits):
			p=softmax(logit)
			surprisal+=-1*np.log(p[token_id])
		all_surprisal.append(surprisal)
	return all_surprisal

def ceiling_division(n, d):
	return -(n // -d)

def delay_one(mat, d):
    # delays a matrix by a delay d. Positive d ==> row t has row t-d
    new_mat = np.zeros_like(mat)
    if d > 0:
        new_mat[d:] = mat[:-d]
    elif d < 0:
        new_mat[:d] = mat[-d:]
    else:
        new_mat = mat
    return new_mat


def delay_mat(mat, delays):
    # delays a matrix by a set of delays d.
    # a row t in the returned matrix has the concatenated:
    # row(t-delays[0],t-delays[1]...t-delays[last] )
    new_mat = np.concatenate([delay_one(mat, d) for d in delays], axis=-1)
    return new_mat

###############################
def Fisher_method(all_pvalues):
	chi=list()
	chi_pvalues=list()
	for pvalues in all_pvalues.T:
		chi_squared=-2*np.sum([np.log(p) for p in pvalues])
		chi_p=chi2.sf(chi_squared,2*len(pvalues))
		chi.append(chi_squared)
		chi_pvalues.append(chi_p)
	return np.array(chi),np.array(chi_pvalues)

def do_ridge_regression(features,brain_responses,n_splits=10):
	kf = KFold(n_splits=n_splits)

	brain_responses_pred=np.zeros(brain_responses.shape)
	for train, test in kf.split(features):
		X_train, X_test, y_train, y_test = features[train], features[test], brain_responses[train], brain_responses[test]

		alphas = np.logspace(-1, 2, 2) # Equally log-spaced alphas between the first number and the second number. The third number is the number of alphas to test.
		nboots = 1 # Number of cross-validation runs.
		chunklen = 40 # 
		nchunks = 20

		wt, corr, alphas, bscorrs, valinds = bootstrap_ridge(X_train, y_train, X_test, y_test,
															alphas, nboots, chunklen, nchunks,
															singcutoff=1e-10, single_alpha=True)
		y_test_pred = np.dot(X_test, wt)

		# fill prediction in the correct position
		for i,word_idx in enumerate(test):
			brain_responses_pred[word_idx] = y_test_pred[i]

	return brain_responses_pred

def cosine_similarity(A,B):
	return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))


def perm_test(D,P1,P2,n=1000):
	mat=np.zeros((D.shape[2],D.shape[0]))
	for ch in range(D.shape[2]):
		print(ch,"...")
		for t in range(D.shape[0]):
			diff_true=pearsonr(D[t,:,ch], P1[t,:,ch])[0] - pearsonr(D[t,:,ch], P2[t,:,ch])[0]
			diff_list=list()
			for i in range(n):
				D_i=np.random.permutation(D[t,:,ch])
				diff_i=pearsonr(D_i, P1[t,:,ch])[0] - pearsonr(D_i, P2[t,:,ch])[0]
				diff_list.append(diff_i)
			p_value=(np.sum([item>diff_true for item in diff_list])+1)/(len(diff_list)+1)
			mat[ch,t]=p_value
	return mat

def FDR_correction(pvals):
	pvals=np.array(pvals)
	if len(pvals.shape)>1:
		pvals_reshape=np.reshape(pvals,np.product(pvals.shape))
	else:
		pvals_reshape=pvals
	pvals_corrected=smm.fdrcorrection(pvals_reshape)[1]
	pvals_corrected=np.reshape(pvals_corrected,pvals.shape)
	return pvals_corrected
###############################

def plot_meg_to_cortex(sensor_file, corrs, fig, ax, file=False, trim_sensor=True):
	"""
	sensor_file: a file that has all information about MEG
	corr_file: (path to a file containing) correlations of channels
	"""

	raw_fname = os.path.join(sensor_file)
	raw = read_raw_ctf(raw_fname,verbose=False)

	# drop suffix
	if trim_sensor:
		new_channel_names=dict()
		for name in raw.info['ch_names']:
			new_name=name.split("-")[0]
			new_channel_names[name]=new_name
		
		raw.rename_channels(new_channel_names,verbose=False)
	
	temp=mne.channel_indices_by_type(raw.info)
	mag_info=mne.pick_info(raw.info,temp['mag'])

	if file:
		corrs=pickle.load(open(corrs,"rb"))
	# print(np.mean(corrs))

	# im,cm=mne.viz.plot_topomap(corrs,mag_info,show=False,sphere=0.2,axes=ax)
	im,cm=mne.viz.plot_topomap(corrs,mag_info,show=False,axes=ax)

	# manually fiddle the position of colorbar
	ax_x_start = 0.95
	ax_x_width = 0.04
	ax_y_start = 0.1
	ax_y_height = 0.9
	cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
	clb = fig.colorbar(im, cax=cbar_ax)

def plot_meg_to_cortex_multiple(sensor_file, corrs_list, fig, axes, file=False, trim_sensor=True):
	"""
	sensor_file: a file that has all information about MEG
	corr_file: (path to a file containing) correlations of channels
	"""

	raw_fname = os.path.join(sensor_file)
	raw = read_raw_ctf(raw_fname,verbose=False)

	# drop suffix
	if trim_sensor:
		new_channel_names=dict()
		for name in raw.info['ch_names']:
			new_name=name.split("-")[0]
			new_channel_names[name]=new_name
		
		raw.rename_channels(new_channel_names,verbose=False)
	
	temp=mne.channel_indices_by_type(raw.info)
	mag_info=mne.pick_info(raw.info,temp['mag'])

	if file:
		corrs=pickle.load(open(corrs,"rb"))
	# print(np.mean(corrs))

	assert len(corrs_list)==axes

	for i,ax in enumerate(axes):
		im,cm=mne.viz.plot_topomap(corrs,mag_info,show=False,axes=axes)

	# manually fiddle the position of colorbar
	ax_x_start = 0.95
	ax_x_width = 0.04
	ax_y_start = 0.1
	ax_y_height = 0.9
	cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
	clb = fig.colorbar(im, cax=cbar_ax)


# def load_glove(file_path):
# 	"""
# 	file_path: a txt file that stores glove embeddings
	
# 	Returns
# 	vocab_npa: all words
# 	embs_npa: a matrix that contains embeddings of words in the same order as vocab_npa
# 	"""
# 	vocab,embeddings = [],[]
	
# 	with open(file_path,'rt') as fi:
# 		full_content = fi.read().strip().split('\n')
	
# 	for i in range(len(full_content)):
# 		i_word = full_content[i].split(' ')[0]
# 		i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
# 		vocab.append(i_word)
# 		embeddings.append(i_embeddings)
	
# 	vocab_npa = np.array(vocab)
# 	embs_npa = np.array(embeddings)

# 	#insert '<pad>' and '<unk>' tokens at start of vocab_npa.
# 	vocab_npa = np.insert(vocab_npa, 0, '<pad>')
# 	vocab_npa = np.insert(vocab_npa, 1, '<unk>')
# 	#print(vocab_npa[:10])

# 	pad_emb_npa = np.zeros((1,embs_npa.shape[1]))   #embedding for '<pad>' token.
# 	unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)    #embedding for '<unk>' token.

# 	#insert embeddings for pad and unk tokens at top of embs_npa.
# 	embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))
# 	#print(embs_npa.shape)

# 	return vocab_npa, embs_npa

In [ ]:
#import scipy.stats
import random
def zscore(mat, return_unzvals=False):
    """Z-scores the rows of [mat] by subtracting off the mean and dividing
    by the standard deviation.
    If [return_unzvals] is True, a matrix will be returned that can be used
    to return the z-scored values to their original state.
    """
    zmat = np.empty(mat.shape, mat.dtype)
    unzvals = np.zeros((zmat.shape[0], 2), mat.dtype)
    for ri in range(mat.shape[0]):
        unzvals[ri,0] = np.std(mat[ri,:])
        unzvals[ri,1] = np.mean(mat[ri,:])
        zmat[ri,:] = (mat[ri,:]-unzvals[ri,1]) / (1e-10+unzvals[ri,0])
    
    if return_unzvals:
        return zmat, unzvals
    
    return zmat

def center(mat, return_uncvals=False):
    """Centers the rows of [mat] by subtracting off the mean, but doesn't 
    divide by the SD.
    Can be undone like zscore.
    """
    cmat = np.empty(mat.shape)
    uncvals = np.ones((mat.shape[0], 2))
    for ri in range(mat.shape[0]):
        uncvals[ri,1] = np.mean(mat[ri,:])
        cmat[ri,:] = mat[ri,:]-uncvals[ri,1]
    
    if return_uncvals:
        return cmat, uncvals
    
    return cmat

def unzscore(mat, unzvals):
    """Un-Z-scores the rows of [mat] by multiplying by unzvals[:,0] (the standard deviations)
    and then adding unzvals[:,1] (the row means).
    """
    unzmat = np.empty(mat.shape)
    for ri in range(mat.shape[0]):
        unzmat[ri,:] = mat[ri,:]*(1e-10+unzvals[ri,0])+unzvals[ri,1]
    return unzmat

def ridge(A, b, alpha):
    """Performs ridge regression, estimating x in Ax=b with a regularization
    parameter of alpha.
    With $G=\alpha I(m_A)$, this function returns $W$ with:
    $W=(A^TA+G^TG)^{-1}A^Tb^T$
    Tantamount to minimizing $||Ax-b||+||\alpha I||$.
    """
    G = np.matrix(np.identity(A.shape[1]) * alpha)
    return np.dot(np.dot(np.linalg.inv(np.dot(A.T,A) + np.dot(G.T,G)), A.T), b.T)

def model_voxels(Rstim, Pstim, Rresp, Presp, alpha):
    """Use ridge regression with regularization parameter [alpha] to model [Rresp]
    using [Rstim].  Correlation coefficients on the test set ([Presp] and [Pstim])
    will be returned for each voxel, as well as the linear weights.
    """
    print ("Z-scoring stimuli (with a flip)... (or not)")
    #zRstim = zscore(Rstim.T).T
    #zPstim = zscore(Pstim.T).T
    
    Rresp[np.isnan(Rresp)] = 0.0
    Presp[np.isnan(Presp)] = 0.0
    
    print ("Running ridge regression...")
    rwts = ridge(Rstim, Rresp.T, alpha)
    print ("Finding correlations...")
    pred = np.dot(Pstim, rwts)
    prednorms = np.apply_along_axis(np.linalg.norm, 0, pred)
    respnorms = np.apply_along_axis(np.linalg.norm, 0, Presp)
    correlations = np.array(np.sum(np.multiply(Presp, pred), 0)).squeeze()/(prednorms*respnorms)
    
    print ("Max correlation: %0.3f" % np.max(correlations))
    print ("Skewness: %0.3f" % scipy.stats.skew(correlations))
    return np.array(correlations), rwts

def model_voxels_old(Rstim, Pstim, Rresp, Presp, alpha):
    """Use ridge regression with regularization parameter [alpha] to model [Rresp]
    using [Rstim].  Correlation coefficients on the test set ([Presp] and [Pstim])
    will be returned for each voxel, as well as the linear weights.
    """
    print ("Z-scoring stimuli (with a flip)...")
    #zRstim = zscore(Rstim.T).T
    #zPstim = zscore(Pstim.T).T
    
    Rresp[np.isnan(Rresp)] = 0.0
    Presp[np.isnan(Presp)] = 0.0
    
    print ("Running ridge regression...")
    rwts = ridge(Rstim, Rresp.T, alpha)
    print ("Finding correlations...")
    correlations = []
    for vi in range(Presp.shape[1]):
        rcorr = np.corrcoef(Presp[:,vi].T,np.array((np.matrix(Pstim) * np.matrix(rwts[:,vi]))).T)[0,1]
        correlations.append(rcorr)
        
    print ("Max correlation: %0.3f" % np.max(correlations))
    print ("Skewness: %0.3f" % scipy.stats.skew(correlations))
    return np.array(correlations), rwts

def gaussianize(vec):
    """Uses a look-up table to force the values in [vec] to be gaussian."""
    ranks = np.argsort(np.argsort(vec))
    cranks = (ranks+1).astype(float)/(ranks.max()+2)
    vals = scipy.stats.norm.isf(1-cranks)
    zvals = vals/vals.std()
    return zvals

def gaussianize_mat(mat):
    """Gaussianizes each column of [mat]."""
    gmat = np.empty(mat.shape)
    for ri in range(mat.shape[1]):
        gmat[:,ri] = gaussianize(mat[:,ri])
    return gmat

def make_delayed(stim, delays, circpad=False):
    """Creates non-interpolated concatenated delayed versions of [stim] with the given [delays] 
    (in samples).
    
    If [circpad], instead of being padded with zeros, [stim] will be circularly shifted.
    """
    nt,ndim = stim.shape
    dstims = []
    for di,d in enumerate(delays):
        dstim = np.zeros((nt, ndim))
        if d<0: ## negative delay
            dstim[:d,:] = stim[-d:,:]
            if circpad:
                dstim[d:,:] = stim[:-d,:]
        elif d>0:
            dstim[d:,:] = stim[:-d,:]
            if circpad:
                dstim[:d,:] = stim[-d:,:]
        else: ## d==0
            dstim = stim.copy()
        dstims.append(dstim)
    return np.hstack(dstims)

def mult_diag(d, mtx, left=True):
    """Multiply a full matrix by a diagonal matrix.
    This function should always be faster than dot.

    Input:
      d -- 1D (N,) array (contains the diagonal elements)
      mtx -- 2D (N,N) array

    Output:
      mult_diag(d, mts, left=True) == dot(diag(d), mtx)
      mult_diag(d, mts, left=False) == dot(mtx, diag(d))
    
    By Pietro Berkes
    From http://mail.scipy.org/pipermail/numpy-discussion/2007-March/026807.html
    """
    if left:
        return (d*mtx.T).T
    else:
        return d*mtx

import time
import logging
def counter(iterable, countevery=100, total=None, logger=logging.getLogger("counter")):
    """Logs a status and timing update to [logger] every [countevery] draws from [iterable].
    If [total] is given, log messages will include the estimated time remaining.
    """
    start_time = time.time()

    ## Check if the iterable has a __len__ function, use it if no total length is supplied
    if total is None:
        if hasattr(iterable, "__len__"):
            total = len(iterable)
    
    for count, thing in enumerate(iterable):
        yield thing
        
        if not count%countevery:
            current_time = time.time()
            rate = float(count+1)/(current_time-start_time)

            if rate>1: ## more than 1 item/second
                ratestr = "%0.2f items/second"%rate
            else: ## less than 1 item/second
                ratestr = "%0.2f seconds/item"%(rate**-1)
            
            if total is not None:
                remitems = total-(count+1)
                remtime = remitems/rate
                timestr = ", %s remaining" % time.strftime('%H:%M:%S', time.gmtime(remtime))
                itemstr = "%d/%d"%(count+1, total)
            else:
                timestr = ""
                itemstr = "%d"%(count+1)

            formatted_str = "%s items complete (%s%s)"%(itemstr,ratestr,timestr)
            if logger is None:
                print (formatted_str)
            else:
                logger.info(formatted_str)


def wait_for_disk(dir, maxtime=0.2, retrytime=10.0, maxtries=100):
    """Waits to continue until disk is not slammed.
    """
    for trynum in range(maxtries):
        stime = time.time()
        os.listdir(dir)
        lstime = time.time() - stime
        if lstime < maxtime:
            print ("Disk access is quick (%0.3f seconds to ls), continuing.." % lstime)
            return
        else:
            print ("Disk access is slow (%0.3f seconds to ls), waiting more.." % lstime)
            time.sleep(retrytime)

    print ("Disk access is slow but I'm starting anyway..")

# Ridge Correlation

In [ ]:
#import scipy
from functools import reduce
import logging
# from .utils import mult_diag, counter
import random
import itertools as itools

zs = lambda v: (v-v.mean(0))/v.std(0) ## z-score function


def ridge_corr(Rstim, Pstim, Rresp, Presp, alphas, normalpha=False, dtype=np.single, corrmin=0.2,
               singcutoff=1e-10, use_corr=True, logger=logging.getLogger("ridge_corr")):
    """Uses ridge regression to find a linear transformation of [Rstim] that approximates [Rresp].
    Then tests by comparing the transformation of [Pstim] to [Presp]. This procedure is repeated
    for each regularization parameter alpha in [alphas]. The correlation between each prediction and
    each response for each alpha is returned. Note that the regression weights are NOT returned.

    Parameters
    ----------
    Rstim : array_like, shape (TR, N)
        Training stimuli with TR time points and N features. Each feature should be Z-scored across time.
    Pstim : array_like, shape (TP, N)
        Test stimuli with TP time points and N features. Each feature should be Z-scored across time.
    Rresp : array_like, shape (TR, M)
        Training responses with TR time points and M responses (voxels, neurons, what-have-you).
        Each response should be Z-scored across time.
    Presp : array_like, shape (TP, M)
        Test responses with TP time points and M responses.
    alphas : list or array_like, shape (A,)
        Ridge parameters to be tested. Should probably be log-spaced. np.logspace(0, 3, 20) works well.
    normalpha : boolean
        Whether ridge parameters should be normalized by the Frobenius norm of Rstim. Good for
        comparing models with different numbers of parameters.
    dtype : np.dtype
        All data will be cast as this dtype for computation. np.single is used by default for memory
        efficiency.
    corrmin : float in [0..1]
        Purely for display purposes. After each alpha is tested, the number of responses with correlation
        greater than corrmin minus the number of responses with correlation less than negative corrmin
        will be printed. For long-running regressions this vague metric of non-centered skewness can
        give you a rough sense of how well the model is working before it's done.
    singcutoff : float
        The first step in ridge regression is computing the singular value decomposition (SVD) of the
        stimulus Rstim. If Rstim is not full rank, some singular values will be approximately equal
        to zero and the corresponding singular vectors will be noise. These singular values/vectors
        should be removed both for speed (the fewer multiplications the better!) and accuracy. Any
        singular values less than singcutoff will be removed.
    use_corr : boolean
        If True, this function will use correlation as its metric of model fit. If False, this function
        will instead use variance explained (R-squared) as its metric of model fit. For ridge regression
        this can make a big difference -- highly regularized solutions will have very small norms and
        will thus explain very little variance while still leading to high correlations, as correlation
        is scale-free while R**2 is not.

    Returns
    -------
    Rcorrs : array_like, shape (A, M)
        The correlation between each predicted response and each column of Presp for each alpha.
    
    """
    ## Calculate SVD of stimulus matrix
    logger.info("Doing SVD...")
    try:
        U,S,Vh = np.linalg.svd(Rstim, full_matrices=False)
    except np.linalg.LinAlgError as e:
        logger.info("NORMAL SVD FAILED, trying more robust dgesvd..")
        from text.regression.svd_dgesvd import svd_dgesvd
        U,S,Vh = svd_dgesvd(Rstim, full_matrices=False)

    ## Truncate tiny singular values for speed
    origsize = S.shape[0]
    ngoodS = np.sum(S>singcutoff)
    nbad = origsize-ngoodS
    U = U[:,:ngoodS]
    S = S[:ngoodS]
    Vh = Vh[:ngoodS]
    logger.info("Dropped %d tiny singular values.. (U is now %s)"%(nbad, str(U.shape)))

    ## Normalize alpha by the Frobenius norm
    #frob = np.sqrt((S**2).sum()) ## Frobenius!
    frob = S[0]
    #frob = S.sum()
    logger.info("Training stimulus has Frobenius norm: %0.03f"%frob)
    if normalpha:
        nalphas = alphas * frob
    else:
        nalphas = alphas

    ## Precompute some products for speed
    UR = np.dot(U.T, Rresp) ## Precompute this matrix product for speed
    PVh = np.dot(Pstim, Vh.T) ## Precompute this matrix product for speed
    
    #Prespnorms = np.apply_along_axis(np.linalg.norm, 0, Presp) ## Precompute test response norms
    zPresp = zs(Presp)
    Prespvar = Presp.var(0)
    Rcorrs = [] ## Holds training correlations for each alpha
    for na, a in zip(nalphas, alphas):
        #D = np.diag(S/(S**2+a**2)) ## Reweight singular vectors by the ridge parameter 
        D = S/(S**2+na**2) ## Reweight singular vectors by the (normalized?) ridge parameter
        
        pred = np.dot(mult_diag(D, PVh, left=False), UR) ## Best (1.75 seconds to prediction in test)
        # pred = np.dot(mult_diag(D, np.dot(Pstim, Vh.T), left=False), UR) ## Better (2.0 seconds to prediction in test)
        
        # pvhd = reduce(np.dot, [Pstim, Vh.T, D]) ## Pretty good (2.4 seconds to prediction in test)
        # pred = np.dot(pvhd, UR)
        
        # wt = reduce(np.dot, [Vh.T, D, UR]).astype(dtype) ## Bad (14.2 seconds to prediction in test)
        # wt = reduce(np.dot, [Vh.T, D, U.T, Rresp]).astype(dtype) ## Worst
        # pred = np.dot(Pstim, wt) ## Predict test responses

        if use_corr:
            #prednorms = np.apply_along_axis(np.linalg.norm, 0, pred) ## Compute predicted test response norms
            #Rcorr = np.array([np.corrcoef(Presp[:,ii], pred[:,ii].ravel())[0,1] for ii in range(Presp.shape[1])]) ## Slowly compute correlations
            #Rcorr = np.array(np.sum(np.multiply(Presp, pred), 0)).squeeze()/(prednorms*Prespnorms) ## Efficiently compute correlations
            Rcorr = (zPresp*zs(pred)).mean(0)
        else:
            ## Compute variance explained
            resvar = (Presp-pred).var(0)
            Rcorr = np.clip(1-(resvar/Prespvar), 0, 1)
            
        Rcorr[np.isnan(Rcorr)] = 0
        Rcorrs.append(Rcorr)
        
        log_template = "Training: alpha=%0.3f, mean corr=%0.5f, max corr=%0.5f, over-under(%0.2f)=%d"
        log_msg = log_template % (a,
                                  np.mean(Rcorr),
                                  np.max(Rcorr),
                                  corrmin,
                                  (Rcorr>corrmin).sum()-(-Rcorr>corrmin).sum())
        if logger is not None:
            logger.info(log_msg)
        else:
            print (log_msg)
    
    return Rcorrs


def bootstrap_ridge(Rstim, Rresp, Pstim, Presp, alphas, nboots, chunklen, nchunks, dtype=np.single,
                    corrmin=0.2, joined=None, singcutoff=1e-10, normalpha=False, single_alpha=False,
                    use_corr=True, logger=logging.getLogger("ridge_corr")):
    """Uses ridge regression with a bootstrapped held-out set to get optimal alpha values for each response.
    [nchunks] random chunks of length [chunklen] will be taken from [Rstim] and [Rresp] for each regression
    run.  [nboots] total regression runs will be performed.  The best alpha value for each response will be
    averaged across the bootstraps to estimate the best alpha for that response.
    
    If [joined] is given, it should be a list of lists where the STRFs for all the voxels in each sublist 
    will be given the same regularization parameter (the one that is the best on average).
    
    Parameters
    ----------
    Rstim : array_like, shape (TR, N)
        Training stimuli with TR time points and N features. Each feature should be Z-scored across time.
    Rresp : array_like, shape (TR, M)
        Training responses with TR time points and M different responses (voxels, neurons, what-have-you).
        Each response should be Z-scored across time.
    Pstim : array_like, shape (TP, N)
        Test stimuli with TP time points and N features. Each feature should be Z-scored across time.
    Presp : array_like, shape (TP, M)
        Test responses with TP time points and M different responses. Each response should be Z-scored across
        time.
    alphas : list or array_like, shape (A,)
        Ridge parameters that will be tested. Should probably be log-spaced. np.logspace(0, 3, 20) works well.
    nboots : int
        The number of bootstrap samples to run. 15 to 30 works well.
    chunklen : int
        On each sample, the training data is broken into chunks of this length. This should be a few times 
        longer than your delay/STRF. e.g. for a STRF with 3 delays, I use chunks of length 10.
    nchunks : int
        The number of training chunks held out to test ridge parameters for each bootstrap sample. The product
        of nchunks and chunklen is the total number of training samples held out for each sample, and this 
        product should be about 20 percent of the total length of the training data.
    dtype : np.dtype
        All data will be cast as this dtype for computation. np.single is used by default for memory efficiency,
        as using np.double will thrash most machines on a big problem. If you want to do regression on 
        complex variables, this should be changed to np.complex128.
    corrmin : float in [0..1]
        Purely for display purposes. After each alpha is tested for each bootstrap sample, the number of 
        responses with correlation greater than this value will be printed. For long-running regressions this
        can give a rough sense of how well the model works before it's done.
    joined : None or list of array_like indices
        If you want the STRFs for two (or more) responses to be directly comparable, you need to ensure that
        the regularization parameter that they use is the same. To do that, supply a list of the response sets
        that should use the same ridge parameter here. For example, if you have four responses, joined could
        be [np.array([0,1]), np.array([2,3])], in which case responses 0 and 1 will use the same ridge parameter
        (which will be parameter that is best on average for those two), and likewise for responses 2 and 3.
    singcutoff : float
        The first step in ridge regression is computing the singular value decomposition (SVD) of the
        stimulus Rstim. If Rstim is not full rank, some singular values will be approximately equal
        to zero and the corresponding singular vectors will be noise. These singular values/vectors
        should be removed both for speed (the fewer multiplications the better!) and accuracy. Any
        singular values less than singcutoff will be removed.
    normalpha : boolean
        Whether ridge parameters (alphas) should be normalized by the Frobenius norm of Rstim. Good for rigorously
        comparing models with different numbers of parameters.
    single_alpha : boolean
        Whether to use a single alpha for all responses. Good for identification/decoding.
    use_corr : boolean
        If True, this function will use correlation as its metric of model fit. If False, this function
        will instead use variance explained (R-squared) as its metric of model fit. For ridge regression
        this can make a big difference -- highly regularized solutions will have very small norms and
        will thus explain very little variance while still leading to high correlations, as correlation
        is scale-free while R**2 is not.
    
    Returns
    -------
    wt : array_like, shape (N, M)
        Regression weights for N features and M responses.
    corrs : array_like, shape (M,)
        Validation set correlations. Predicted responses for the validation set are obtained using the regression
        weights: pred = np.dot(Pstim, wt), and then the correlation between each predicted response and each 
        column in Presp is found.
    alphas : array_like, shape (M,)
        The regularization coefficient (alpha) selected for each voxel using bootstrap cross-validation.
    bootstrap_corrs : array_like, shape (A, M, B)
        Correlation between predicted and actual responses on randomly held out portions of the training set,
        for each of A alphas, M voxels, and B bootstrap samples.
    valinds : array_like, shape (TH, B)
        The indices of the training data that were used as "validation" for each bootstrap sample.
    """
    nresp, nvox = Rresp.shape
    bestalphas = np.zeros((nboots, nvox))  ## Will hold the best alphas for each voxel
    valinds = [] ## Will hold the indices into the validation data for each bootstrap
    
    Rcmats = []
    for bi in counter(range(nboots), countevery=1, total=nboots):
        logger.info("Selecting held-out test set..")
        allinds = range(nresp)
        indchunks = list(zip(*[iter(allinds)]*chunklen))
        random.shuffle(indchunks)
        heldinds = list(itools.chain(*indchunks[:nchunks]))
        notheldinds = list(set(allinds)-set(heldinds))
        valinds.append(heldinds)
        
        RRstim = Rstim[notheldinds,:]
        PRstim = Rstim[heldinds,:]
        RRresp = Rresp[notheldinds,:]
        PRresp = Rresp[heldinds,:]
        
        ## Run ridge regression using this test set
        Rcmat = ridge_corr(RRstim, PRstim, RRresp, PRresp, alphas,
                           dtype=dtype, corrmin=corrmin, singcutoff=singcutoff,
                           normalpha=normalpha, use_corr=use_corr)
        
        Rcmats.append(Rcmat)
    
    ## Find weights for each voxel
    try:
        U,S,Vh = np.linalg.svd(Rstim, full_matrices=False)
    except np.linalg.LinAlgError as e:
        logger.info("NORMAL SVD FAILED, trying more robust dgesvd..")
        from text.regression.svd_dgesvd import svd_dgesvd
        U,S,Vh = svd_dgesvd(Rstim, full_matrices=False)

    ## Normalize alpha by the Frobenius norm
    #frob = np.sqrt((S**2).sum()) ## Frobenius!
    frob = S[0]
    #frob = S.sum()
    logger.info("Total training stimulus has Frobenius norm: %0.03f"%frob)
    if normalpha:
        nalphas = alphas * frob
    else:
        nalphas = alphas

    allRcorrs = np.dstack(Rcmats)
    if not single_alpha:
        logger.info("Finding best alpha for each response..")
        if joined is None:
            ## Find best alpha for each voxel
            meanbootcorrs = allRcorrs.mean(2)
            bestalphainds = np.argmax(meanbootcorrs, 0)
            valphas = nalphas[bestalphainds]
        else:
            ## Find best alpha for each group of voxels
            valphas = np.zeros((nvox,))
            for jl in joined:
                jcorrs = allRcorrs[:,jl,:].mean(1).mean(1) ## Mean across voxels in the set, then mean across bootstraps
                bestalpha = np.argmax(jcorrs)
                valphas[jl] = nalphas[bestalpha]
    else:
        logger.info("Finding single best alpha..")
        meanbootcorr = allRcorrs.mean(2).mean(1)
        bestalphaind = np.argmax(meanbootcorr)
        bestalpha = alphas[bestalphaind]
        valphas = np.array([bestalpha]*nvox)
        logger.info("Best alpha = %0.3f"%bestalpha)

    logger.info("Computing weights for each response using entire training set..")
    UR = np.dot(U.T, np.nan_to_num(Rresp))
    pred = np.zeros(Presp.shape)
    wt = np.zeros((Rstim.shape[1], Rresp.shape[1]))
    for ai,alpha in enumerate(nalphas):
        selvox = np.nonzero(valphas==alpha)[0]
        awt = reduce(np.dot, [Vh.T, np.diag(S/(S**2+alpha**2)), UR[:,selvox]])
        pred[:,selvox] = np.dot(Pstim, awt)
        wt[:,selvox] = awt

    ## Find test correlations
    nnpred = np.nan_to_num(pred)
    corrs = np.nan_to_num(np.array([np.corrcoef(Presp[:,ii], nnpred[:,ii].ravel())[0,1] for ii in range(Presp.shape[1])]))

    return wt, corrs, valphas, allRcorrs, valinds

# MEG Sensor plot

In [ ]:
import os

home=os.path.expanduser("~")

def topoplot(
    mat,
    nrow=4,
    ncol=5,
    time_step=25,
    time_start=0,
    cmap="RdBu_r",
    vmin=-0.1,
    vmax=0.1,
    figsize=(15, 15),
    fontsize=16,
):
    """Creates helmet plots for sensor-space MEG data (based on MNE visualization)

    Args:
        mat (2d numpy array): data to plot of size N sensors x M timepoints
        nrow (int, optional): number of rows in plot. Defaults to 4.
        ncol (int, optional): number of columns in plot. Defaults to 5.
        time_step (int, optional): time window length. Defaults to 25.
        time_start (int, optional): what time to start plotting. Defaults to 0.
        cmap (str, optional): colormap name. Defaults to 'RdBu_r'.
        vmin (float, optional): sets the colorbar min. Defaults to -0.1.
        vmax (float, optional): sets the colorbar max. Defaults to 0.1.
        figsize (tuple, optional): figure size. Defaults to (15,15).
        fontsize (int, optional): font size. Defaults to 16.

    Returns:
        figure handle
    """

    from sklearn.metrics.pairwise import euclidean_distances
    import csv
    import numpy as np
    import mne
    import matplotlib.pyplot as plt

    # gets sensor locations
    with open("../input/extra-divergence-meg-data/locations.txt", "r") as f:
        locs = csv.reader(f, delimiter=",")
        loc306 = np.asarray(
            [[float(w1[0].split(" ")[1]), float(w1[0].split(" ")[2])] for w1 in locs]
        )
    loc102 = loc306[::3]
    loc = {306: loc306, 102: loc102}[mat.shape[0]]  # pick the correct channel locations

    fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=figsize)
    i = 0
    for row in ax:
        for col in row:
            if i < mat.shape[1]:
                h = mne.viz.plot_topomap(
                    mat[:, i],
                    loc,
                    vlim=(vmin,vmax),
                    axes=col,
                    cmap=cmap,
                    show=False,
                    contours = 0,
                )
            i += 1
    i = 0
    for row in ax:
        for col in row:
            col.set_title(
                "{} to {} ms".format(
                    i * time_step + time_start, (i + 1) * time_step + time_start
                ),
                fontsize=fontsize,
            )
            i += 1

    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([1.00, 0.15, 0.01, 0.71])
    cbar = fig.colorbar(h[0], cax=cbar_ax)
    cbar.ax.tick_params(labelsize=fontsize)

    # set the font type and size of the colorbar
    for l in cbar.ax.yaxis.get_ticklabels():
        l.set_size(20)

    plt.tight_layout()
    return fig

def single_topoplot(
    mat,
    cmap="RdBu_r",
    vmin=-0.1,
    vmax=0.1,
    figsize=(5,5),
    fontsize=16,
):
    """Creates single helmet plot for sensor-space MEG data (based on MNE visualization)

    Args:
        mat (2d numpy array): data to plot of size N sensors x M timepoints
        cmap (str, optional): colormap name. Defaults to 'RdBu_r'.
        vmin (float, optional): sets the colorbar min. Defaults to -0.1.
        vmax (float, optional): sets the colorbar max. Defaults to 0.1.
        figsize (tuple, optional): figure size. Defaults to (15,15).
        fontsize (int, optional): font size. Defaults to 16.

    Returns:
        figure handle
    """

    from sklearn.metrics.pairwise import euclidean_distances
    import csv
    import numpy as np
    import mne
    import matplotlib.pyplot as plt

    # gets sensor locations
    with open("../input/extra-divergence-meg-data/locations.txt", "r") as f:
        locs = csv.reader(f, delimiter=",")
        loc306 = np.asarray(
            [[float(w1[0].split(" ")[1]), float(w1[0].split(" ")[2])] for w1 in locs]
        )
    loc102 = loc306[::3]
    loc = {306: loc306, 102: loc102}[mat.shape[0]]  # pick the correct channel locations

    fig = plt.figure(figsize=figsize)
    h = mne.viz.plot_topomap(
                    mat,
                    loc,
                    vlim=(vmin,vmax),
                    cmap=cmap,
                    show=False,
                    size=5,
                )

    fig.subplots_adjust(right=0.7)
    cbar_ax = fig.add_axes([1.05, 0.15, 0.05, 0.7])
    cbar = fig.colorbar(h[0], cax=cbar_ax)
    cbar.ax.tick_params(labelsize=fontsize)

    # set the font type and size of the colorbar
    for l in cbar.ax.yaxis.get_ticklabels():
        l.set_size(20)

    plt.tight_layout()
    return fig

# Generate LM Embedding for HP

In [ ]:
# Generate LM Embedding for HP
import os
import sys
import numpy as np
import string
import pickle
import argparse

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import zscore
import scipy.io as sio
from scipy.io import loadmat

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm

from transformers import AutoTokenizer, AutoModelForCausalLM

# from mat4py import loadmat

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../")
# from dataloader.dataloader import HP_dataset_denoised

# from utils import utils
# from utils.ridge.ridge import bootstrap_ridge

np.set_printoptions(precision=3,suppress=True)

import warnings
warnings.filterwarnings("ignore")



home=os.path.expanduser("~")

parser = argparse.ArgumentParser()

parser.add_argument("--dataset", default="HP")
parser.add_argument("--base_data_path", default="../input/divergence-meg-data/")
parser.add_argument("--chapter", type=int, default=1)
parser.add_argument("--meg_offset", type=int, default=0)

parser.add_argument("--batch_size", type=int, default=4)
parser.add_argument("--sequence_length", type=int, default=20)
parser.add_argument("--lm_name", default="GPT2-xl")
parser.add_argument("--model_info", default="base")
parser.add_argument("--lm_path",default = "../working/interim_data/lm_embeddings/")
###################################################################################
args_list = [
    "--dataset", "HP",
    "--base_data_path", "../input/divergence-meg-data/",
    "--chapter", "1",
    "--meg_offset", "0",
    "--batch_size", "4",
    "--sequence_length", "20",
    "--lm_name", "GPT2-xl",
    "--model_info", "base",
    "--lm_path","../working/interim_data/lm_embeddings/" # Provide a value for lm_path as it has no default
]

# --- Example of using the args_list ---
print("--- Parsing with provided args_list ---")
args = parser.parse_args(args_list)
###################################################################################
args.no_cuda=not torch.cuda.is_available()

print("Echo arguments:",args)

## Load datasetGPT2
whole_data = HP_dataset_denoised(args)
print("Number of words:", len(whole_data))

## Load LM
if args.model_info=="base":	# if loading base model
    tokenizer,model=load_tokenizer_and_model_from_transformers(args.lm_name)
else:	# if loading finetuned model
    if args.lm_name == "Llama-2":
        tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", truncation_side="left", padding_side="left")
    elif args.lm_name == "GPT2-xl":
        tokenizer = AutoTokenizer.from_pretrained("gpt2-xl", truncation_side="left", padding_side="left")
    else:
        raise ValueError("Not a valid lm_name.")
    model = AutoModelForCausalLM.from_pretrained(args.lm_path, output_hidden_states=True)
    
tokenizer.pad_token = tokenizer.bos_token

if not args.no_cuda:
    model=model.to("cuda:0")
model.eval();


# #################################################--- Begin of new code: narrative categorization setup ---#################################################
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

# Define keyword sets for rough rule-based tagging
dialogue_markers = {'said', 'asked', 'replied', 'shouted'}
emotion_words = {'angry', 'sad', 'happy', 'nervous', 'afraid', 'proud', 'furious', 'pleased'}
action_words = {'walked', 'ran', 'drove', 'left', 'turned', 'put', 'looked', 'came'}
magic_words = {'wand', 'cloak', 'owl', 'spell', 'wizard', 'witch', 'magic'}
description_words = {'Privet', 'Drive', 'long', 'thin', 'square', 'dark', 'sky', 'street'}

def categorize_word(word):
    w = word.lower().strip(string.punctuation)
    if w in dialogue_markers:
        return "dialogue"
    elif w in emotion_words or sia.polarity_scores(w)["compound"] != 0:
        return "emotional"
    elif w in action_words:
        return "action"
    elif w in magic_words:
        return "magic"
    elif w in description_words:
        return "description"
    else:
        return "other"
# #################################################----- End of new code ---#################################################




## Get LM embeddings
dataloader=DataLoader(whole_data, batch_size=args.batch_size)

all_logits=list()
all_embeddings=list()
all_last_words=list()
all_megs=list()
word_categories = []
for batch_i,(batch_text_idx,batch_meg) in enumerate(dataloader):
    batch_text=list()
    batch_last_words=list()
    for text_idx in batch_text_idx:
        line=list()
        for word_idx in text_idx:
            word=whole_data.index_to_word[int(word_idx)]
            line.append(word)
            last_word=word.strip(string.punctuation) # remove punctuations at start and end of the word
        line=" ".join(line)
        # line+=tokenizer.bos_token
        batch_text.append(line)
        batch_last_words.append(last_word)
# #################################################--- Begin of new code: categorize word ---#################################################
        category = categorize_word(last_word)
        word_categories.append(category)
# #################################################--- End of new code ---#################################################

    encodings = tokenizer(batch_text, return_tensors="pt", padding=True)['input_ids']
    # encodings = tokenizer(batch_text, return_tensors="pt"args_list, truncation=True, max_length=args.sequence_length)['input_ids']
    if not args.no_cuda:
        encodings=encodings.to("cuda:0")

    batch_last_word_pos=find_indices_of_last_word_in_batch(tokenizer,batch_last_words,encodings)

    with torch.no_grad():
        output=model(encodings)

        for i,sentence_logits in enumerate(output['logits']):
            word_start,word_end=batch_last_word_pos[i]
            all_logits.append(sentence_logits[word_start-1:word_end].detach().cpu().numpy())
            embeddings=np.vstack([torch.mean(embed[i,word_start:word_end+1],dim=0).detach().cpu().numpy()  for embed in output['hidden_states']])
            all_embeddings.append(embeddings)
            all_last_words.append(encodings[i][word_start:word_end+1].detach().cpu().numpy())
        all_megs.append(batch_meg.detach().cpu().numpy())

    if batch_i%100==0:
        print(f"Batch {batch_i+1}/{len(dataloader)}...")

In [ ]:

all_megs=np.vstack(all_megs)	# (word, meg_channel)
all_embeddings=np.stack(all_embeddings,axis=1)	# (layer, word, embedding_dim)

dumped_data=dict(
        all_megs=all_megs,
        all_embeddings=all_embeddings,
        all_last_words=all_last_words,
        all_logits=all_logits
    )



os.makedirs("../working/interim_data/lm_embeddings/",exist_ok=True)
pickle.dump(dumped_data,
    open(f"../working/interim_data/lm_embeddings/{args.dataset}_chpt{args.chapter}_{args.lm_name}_{args.model_info}.pkl","wb"))
# #################################################--- Begin of new code: save word categories ---#################################################
pickle.dump(word_categories, open("../working/interim_data/lm_embeddings/hp1_word_categories.pkl", "wb"))
# #################################################--- End of new code ---#################################################

In [ ]:
len(word_categories)

In [ ]:
Categories

In [ ]:
# os.makedirs("../working/interim_data/lm_embeddings/",exist_ok=True)
# pickle.dump(dumped_data,
#     open(f"../working/interim_data/lm_embeddings/{args.dataset}_chpt{args.chapter}_{args.lm_name}_{args.model_info}.pkl","wb"))

In [ ]:
os.listdir("../working/interim_data/lm_embeddings/")

In [ ]:
# os.makedirs("../working/interim_data/lm_embeddings/",exist_ok=True)


# Generate data for analysis HP

In [ ]:
#Generate data for analysis HP
import os
import sys
import numpy as np
import string
import pickle
import argparse

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import zscore
import scipy.io as sio
from scipy.io import loadmat

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm

# from mat4py import loadmat

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../")
# from dataloader.dataloader import HP_dataset_denoised
# from utils import meg_sensor_plot
# from utils import utils
# from utils.ridge.ridge import bootstrap_ridge

np.set_printoptions(precision=3,suppress=True)

import warnings
warnings.filterwarnings("ignore")



home=os.path.expanduser("~")

parser = argparse.ArgumentParser()

parser.add_argument("--dataset", default="HP")
parser.add_argument("--base_data_path", default="../input/divergence-meg-data/")
parser.add_argument("--chapter", type=int, default=1)
parser.add_argument("--meg_offset", type=int, default=0)
parser.add_argument("--p_thresh", type=float, default=0.001)

parser.add_argument("--sequence_length", type=int, default=20)
parser.add_argument("--lm_name", default="GPT2-xl")
parser.add_argument("--model_info", default="base")
parser.add_argument("--layer", type=int, default=-1)


# --- FIX STARTS HERE ---
# Simulate command-line arguments by passing a list of strings

# Manually construct the list based on the defaults you want to use
args_list = [
    "--dataset", "HP",
    "--base_data_path", "../input/divergence-meg-data/", # Use the path variable
    "--chapter", "1",
    "--meg_offset", "0",
    "--p_thresh", "0.001", # Convert float to string

    "--sequence_length", "20",
    "--lm_name", "GPT2-xl",
    "--model_info", "base",
    "--layer", "-1"
]

# Pass the list of arguments to parse_args()
args = parser.parse_args(args_list) # <--- The fix is here
# --- FIX ENDS HERE ---


args.lm_embed_path = f"../working/interim_data/lm_embeddings/{args.dataset}_chpt{args.chapter}_{args.lm_name}_{args.model_info}.pkl"

print("Echo arguments:",args)

## Load MEG data
whole_data = HP_dataset_denoised(args)
meg_data=whole_data.all_megs[args.sequence_length:]	# (word, meg_channel, time)

## Load LM embeddings
data=pickle.load(open(args.lm_embed_path,"rb"))
all_embeddings=data["all_embeddings"]	# (layer, word, embedding_dim)

word_num = meg_data.shape[0]
channel_num = meg_data.shape[1]
timepoint_num = meg_data.shape[2]
layer_num = 1

## init arrays
all_corrs = np.zeros((layer_num, timepoint_num, channel_num))
all_sig_channels = np.zeros((layer_num, timepoint_num, channel_num), dtype=int)
all_pred = np.zeros((layer_num, timepoint_num, word_num, channel_num))
all_MSE = np.zeros((layer_num, timepoint_num, word_num))
all_cos_sim = np.zeros((layer_num, timepoint_num, word_num))

#%% ##########################################----NEW----BEGINS----#######################################################
narrative_categories = np.array(word_categories)
categories = np.unique(narrative_categories)
MSEs_for_channels = np.zeros((timepoint_num, channel_num))*np.nan  # shape: (timepoints, channels)
cosine_sim_channels = np.zeros((timepoint_num, channel_num))*np.nan  # shape: (timepoints, channels)
mse_by_category = {tt: {cat: [] for cat in categories} for tt in range(timepoint_num)}
cos_by_category = {tt: {cat: [] for cat in categories} for tt in range(timepoint_num)}
category_word_indices = {cat: np.where(narrative_categories == cat)[0] for cat in categories}
divergence_by_category = {cat: np.zeros((timepoint_num, channel_num)) for cat in categories}

#%% ##########################################----NEW----ENDS----#######################################################
## Ridge regression
## Variable "tt" was "time" in the original code and was changed due to conflict with time module (time.time())
print("Ridge regression...")
for tt in range(timepoint_num):
    brain_responses = meg_data[:,:,tt]
    embeddings = all_embeddings[args.layer]
    brain_responses_pred = do_ridge_regression(embeddings, brain_responses)

    all_pred[0, tt] = brain_responses_pred

    for ch, (pred,actual) in enumerate(zip(brain_responses_pred.T, brain_responses.T)):	# for each channel, all words
        corr, pvalue = pearsonr(pred,actual,alternative="greater")
        all_corrs[0, tt, ch] = corr
        if pvalue < args.p_thresh:
            all_sig_channels[0, tt, ch] = 1

    sig_channels = all_sig_channels[0, tt]
    print(f"layer {args.layer}, {tt*25}-{(tt+1)*25}ms: {np.sum(sig_channels)}")

    for word, (pred,actual) in enumerate(zip(brain_responses_pred, brain_responses)):	# for each word, all sig channels
        all_MSE[0, tt, word] = np.linalg.norm(pred[sig_channels]-actual[sig_channels], ord=2) ** 2 / len(sig_channels)
        all_cos_sim[0, tt, word] = cosine_similarity(pred[sig_channels],actual[sig_channels])


        mse = np.linalg.norm(pred[sig_channels] - actual[sig_channels], ord=2) ** 2 / len(sig_channels)
        cos = cosine_similarity(pred[sig_channels], actual[sig_channels])
#%% ##########################################----NEW----BEGINS----#######################################################
        category = narrative_categories[word]  # 'dialogue', 'action', or 'emotion'

        if category not in categories:
            continue  # skip unknowns
        # ========== GROUP BY CATEGORY ==========
        mse_by_category[tt][category].append(mse)
        cos_by_category[tt][category].append(cos)

        for ch in range(channel_num):
            word_idxs = category_word_indices[category]
            preds = brain_responses_pred[word_idxs, ch]
            actuals = brain_responses[word_idxs, ch]
            mse = np.mean((preds[sig_channels] - actuals[sig_channels]) ** 2)/ len(sig_channels)

            divergence_by_category[category][tt, ch] = mse



    for ch in range(channel_num):
        preds = brain_responses_pred[:, ch]    # shape: (words,) - predictions for one channel across all words
        actuals = brain_responses[:, ch]       # shape: (words,) - actuals for one channel across all words
        MSEs_for_channels[tt, ch] = np.linalg.norm(preds[sig_channels]-actuals[sig_channels], ord=2) ** 2 / len(sig_channels)
        cosine_sim_channels[tt, ch] = cosine_similarity(preds[sig_channels],actuals[sig_channels])
        # divergence_by_category[cat][tt, ch] = mse



        # # MSE for this specific channel across all words
        # mse = np.mean((preds[sig_channels] - actuals[sig_channels]) ** 2) / len(sig_channels)
        # MSEs_for_channels[tt, ch] = mse

        # # Cosine Similarity for this specific channel across all words
        # # Check if norms are non-zero to avoid division by zero in cosine_similarity if any array is all zeros
        # if np.linalg.norm(preds) > 0 and np.linalg.norm(actuals) > 0:
        #     # Reshape to (1, -1) for sklearn.metrics.pairwise.cosine_similarity
        #     cos_sim = cosine_similarity(preds[sig_channels], actuals[sig_channels])
        # else:
        #     cos_sim = np.nan # Define behavior for all-zero vectors

        # cosine_sim_channels[tt, ch] = cos_sim

#%% ##########################################----NEW----ENDS----#######################################################

### Compare Mean Divergence Over Time

In [ ]:
for cat in categories:
    mean_mse = divergence_by_category[cat].mean(axis=1)
    plt.plot(np.arange(timepoint_num)*25,mean_mse, label=cat)

plt.xlabel("Time (ms)")
plt.ylabel("Mean MSE")
plt.legend()
plt.title("Divergence by Narrative Type Over Time")
plt.tight_layout()
plt.show()


### Topomap Per Narrative Type

In [ ]:
import pandas as pd
# Load MEG metadata
meg_channels = pd.read_csv('../input/extra-divergence-meg-data/MEG_Channel_Brain_Areas.csv')
channel_names = meg_channels['Sensor_Label'].tolist()
channel_regions = {
    name: meg_channels.loc[i, 'Brain_Region']
    for i, name in enumerate(channel_names)
}
meg_channels['Sensor_Label'].tolist()
channel_names_mne  = [f'MEG {i:04d}' for i in channel_names]
channel_names_mne[:10]

In [ ]:
info = mne.create_info(ch_names=channel_names_mne,
                       sfreq=1000, # Assuming sfreq remains 1000
                       ch_types=['mag'] * 102 + ['grad'] * 204) # Assuming 102 mag, 204 grad

for cat in categories:
    mean_topo = divergence_by_category[cat].mean(axis=0)
    single_topoplot(mean_topo,vmin = np.min(mean_topo),vmax=np.max(mean_topo))
    plt.title(f"{cat}")



# top-divergent channels

In [ ]:
import pandas as pd
import numpy as np

# Average across time (assuming MSEs_for_channels and cosine_sim_channels are shape [time, 306])
mean_MSE_per_channel = np.mean(MSEs_for_channels, axis=0)         # shape: (306,)
mean_cos_per_channel = np.mean(cosine_sim_channels, axis=0)       # shape: (306,)

# Get top N most divergent channels (highest MSE)
top_n = 10
top_MSE_channels = np.argsort(mean_MSE_per_channel)[-top_n:][::-1]  # descending order
top_MSE = mean_MSE_per_channel[top_MSE_channels]
top_cos_for_top_MSE = mean_cos_per_channel[top_MSE_channels]       # match cosine to same channels

# Load MEG metadata
meg_channels = pd.read_csv('../input/extra-divergence-meg-data/MEG_Channel_Brain_Areas.csv')
channel_names = meg_channels['Sensor_Label'].tolist()
channel_regions = {
    name: meg_channels.loc[i, 'Brain_Region']
    for i, name in enumerate(channel_names)
}

# Print info
print("\nTop channels by HIGHEST MSE:")
for idx, ch_idx in enumerate(top_MSE_channels):
    name = channel_names[ch_idx]
    region = channel_regions[name]
    print(f"{name} - Region: {region}, MSE = {top_MSE[idx]:.4f}, Cos = {top_cos_for_top_MSE[idx]:.4f}")


In [ ]:
single_topoplot(mean_MSE_per_channel,    vmin=np.min(mean_MSE_per_channel),
    vmax=np.max(mean_MSE_per_channel),)
plt.title("Top channels by HIGHEST MSE")

In [ ]:
lowest_cos_channels = np.argsort(mean_cos_per_channel)[:top_n]
lowest_cos = mean_cos_per_channel[lowest_cos_channels]
mse_for_lowest_cos = mean_MSE_per_channel[lowest_cos_channels]

print("\nTop channels by LOWEST Cosine Similarity:")
for idx, ch_idx in enumerate(lowest_cos_channels):
    name = channel_names[ch_idx]
    region = channel_regions[name]
    print(f"{name} - Region: {region}, Cos = {lowest_cos[idx]:.4f}, MSE = {mse_for_lowest_cos[idx]:.4f}")


**In This context (MEG + LM prediction):
We're comparing predicted brain responses and actual brain responses channel-wise.**

- A negative cosine similarity in a channel suggests that:

- The pattern of activation predicted by the language model is inversely correlated with the actual brain signal in that channel.


**Interpretation:**

-  This might mean the model misunderstood the representation for that word/region/timepoint.

- Or the brain region may be involved in processing that is orthogonal or opposing to the language model’s latent representation — possibly reflecting other cognitive processes like:

    - Emotion vs. logic

    - Sensory vs. semantic

    - Inhibition, conflict, etc.
 
  

## Furthur analysis

In [ ]:
# For each word and category, accumulate channel-wise divergence
mse_per_channel_by_category = {cat: np.zeros(channel_num) for cat in categories}
counts = {cat: 0 for cat in categories}

for tt in range(timepoint_num):
    sig_channels = all_sig_channels[0, tt]

    for word, (pred, actual) in enumerate(zip(all_pred[0, tt], meg_data[:, :, tt])):
        category = narrative_categories[word]
        if category not in categories:
            continue

        mse_ch = (pred - actual) ** 2  # shape: (306,)
        mse_per_channel_by_category[category] += mse_ch
        counts[category] += 1

# Normalize
for cat in categories:
    mse_per_channel_by_category[cat] /= counts[cat]


In [ ]:
meg_channels['Sensor_Label'].tolist()
channel_names_mne  = [f'MEG {i:04d}' for i in channel_names]
channel_names_mne[:10]

In [ ]:
mean_MSE_per_channel.shape

In [ ]:
mean_MSE_per_channel

In [ ]:
print(MSEs_for_channels.shape)
print(cosine_sim_channels.shape)
print(brain_responses.shape)


In [ ]:
np.mean((pred[sig_channels]-actual[sig_channels]) ** 2)

In [ ]:
np.linalg.norm(pred[sig_channels]-actual[sig_channels], ord=2) ** 2 / len(sig_channels)

In [ ]:
 for ch in range(channel_num):
    plt.plot(MSEs_for_channels[:,ch])
plt.xlabel("Timepoints")
plt.ylabel("MSE")
plt.legend()
plt.title("Divergence (MSE) per Narrative Category")
plt.show()

In [ ]:
time_points = np.arange(timepoint_num) * 25  # in ms (e.g., 0, 25, ..., 975)

# Plot MSE
plt.figure(figsize=(12, 5))
for cat in categories:
    mean_mse = [np.mean(mse_by_category[tt][cat]) for tt in range(timepoint_num)]
    plt.plot(time_points, mean_mse, label=f"MSE - {cat}")
plt.xlabel("Time after word onset (ms)")
plt.ylabel("Mean Squared Error")
plt.title("Divergence (MSE) over Time by Narrative Category")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot Cosine Similarity
plt.figure(figsize=(12, 5))
for cat in categories:
    mean_cos = [np.mean(cos_by_category[tt][cat]) for tt in range(timepoint_num)]
    plt.plot(time_points, mean_cos, label=f"Cosine Sim - {cat}")
plt.xlabel("Time after word onset (ms)")
plt.ylabel("Cosine Similarity")
plt.title("Cosine Similarity over Time by Narrative Category")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
os.makedirs('../working/data_for_analysis',exist_ok = True)
pickle.dump(
    dict(
    embedding=all_embeddings,
    text=whole_data.all_texts,
    actual=meg_data,
    pred=all_pred,
    corr=all_corrs,
    MSE=all_MSE,
    cos=all_cos_sim,
    sig_channels=all_sig_channels,
    ),
    open(f"../working/data_for_analysis/{args.dataset}_chpt{args.chapter}_{args.lm_name}_{args.model_info}_layer{args.layer}.pkl","wb"))

# Format Sentence

In [ ]:

import os
import sys
import numpy as np
import string
import pickle
import argparse

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import zscore
import scipy.io as sio
from scipy.io import loadmat

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../")
# from dataloader.dataloader import HP_dataset_denoised
# from utils import meg_sensor_plot
# from utils import utils
# from utils.ridge.ridge import bootstrap_ridge

np.set_printoptions(precision=3,suppress=True)

import warnings
warnings.filterwarnings("ignore")

def save_sents(divergent_sents, similar_sents, fname):

	data_dict = {
		"dataset_description": "two chapters from 'Harry Potter and the Sorcerer's Stone'",
		"generation": "the difference between language model and human responses to these sentences",	    
		"target": "which sentences induce different responses for language models and human responses",
		"user": "a literary analyst investigating the characteristics of words",
		"A_desc": "sentences where language models and humans show divergent responses",
		"B_desc": "sentences where language models and humans show similar responses",
		"example_hypotheses": [],
		"split": {
			"research": {"A_samples": [], "B_samples": []},
			"validation": {"A_samples": [], "B_samples": []}
		}
	}

	# train/val split (no test)
	train_split_surp = int(len(divergent_sents) * 0.8)
	train_split_unsurp = int(len(similar_sents) * 0.8)
	divergent_sents_train = divergent_sents[:train_split_surp]
	similar_sents_train = similar_sents[:train_split_unsurp]
	divergent_sents_val = divergent_sents[train_split_surp:]
	similar_sents_val = similar_sents[train_split_unsurp:]

	data_dict["split"]["research"]["A_samples"] = divergent_sents_train
	data_dict["split"]["validation"]["A_samples"] = similar_sents_train
	data_dict["split"]["research"]["B_samples"] = divergent_sents_val
	data_dict["split"]["validation"]["B_samples"] = similar_sents_val

	with open(fname, "wb") as f:
		pickle.dump(data_dict, f)
	
	
def rank_sentences(words, mses, search_strings = [".", "!", "?"]):
	all_sentences = list()
	this_sentence = list()
	mse_all_sentences = list()
	mse_count = 0
	
	for i, word in enumerate(words):
		this_sentence.append(word)
		mse_count += mses[i]
		if any(string in word for string in search_strings) and len(this_sentence) > 3: # end of a sentence
			all_sentences.append(" ".join(this_sentence))
			mse_all_sentences.append(mse_count/len(this_sentence))
			this_sentence = list()
			mse_count = 0
			
	sort_idx = np.argsort(mse_all_sentences)
	return [all_sentences[idx] for idx in sort_idx]



home=os.path.expanduser("~")

parser = argparse.ArgumentParser()

parser.add_argument("--dataset", default="HP")
parser.add_argument("--sequence_length", type=int, default=20)

parser.add_argument("--chapter", type=int, default=1)

parser.add_argument("--layer", type=int, default=-1)

parser.add_argument("--save_path", default="../working/interim_data/divergent_sents/")

parser.add_argument("--lm_name", default= "GPT2-xl")

parser.add_argument("--n_sents", type=int, default=100)

args_list = [
    "--dataset", "HP",
    "--sequence_length", "20",
    "--chapter", "1",
    "--layer", "-1",
    "--save_path", "../working/interim_data/divergent_sents/", # Use the constructed path variable
    "--lm_name", "GPT2-xl",
    "--n_sents", "100"
]

# --- Example of using the args_list ---
print("--- Parsing with default args_list ---")
args = parser.parse_args(args_list)


print("Echo arguments:",args)

if not os.path.exists(f"{args.save_path}formatted_sents_{args.lm_name}_layer{args.layer}_chpt{args.chapter}.pkl"):
    os.makedirs('../working/interim_data/divergent_sents/',exist_ok=True)
    
    ## Load data
    if args.chapter==12:
        # data1=pickle.load(open(f"../working/interim_data/lm_embeddings/HP_chpt1_{args.lm_name}_base.pkl","rb"))
        data1=pickle.load(open(f"../working/data_for_analysis/HP_chpt1_{args.lm_name}_base_layer-1.pkl","rb"))
        words1=data1['text'][args.sequence_length:]
        MSEs1=np.mean(data1['MSE'][args.layer][12:17],axis=0)
        # data2=pickle.load(open(f"../working/interim_data/lm_embeddings/HP_chpt2_{args.lm_name}_base.pkl","rb"))
        data2=pickle.load(open(f"../working/data_for_analysis/HP_chpt2_{args.lm_name}_base_layer-1.pkl","rb"))
        words2=data2['text'][args.sequence_length:]
        MSEs2=np.mean(data2['MSE'][args.layer][12:17],axis=0)
        words=words1+words2
        MSEs=np.concatenate([MSEs1,MSEs2])
    elif args.chapter==1:
        # data=pickle.load(open(f"../working/interim_data/lm_embeddings/HP_chpt1_{args.lm_name}_base.pkl","rb"))
        data=pickle.load(open(f"../working/data_for_analysis/HP_chpt1_{args.lm_name}_base_layer-1.pkl","rb"))
        words=data['text'][args.sequence_length:]
        MSEs=np.mean(data['MSE'][args.layer][12:17],axis=0)
    elif args.chapter==2:
        # data=pickle.load(open(f"../working/interim_data/lm_embeddings/HP_chpt2_{args.lm_name}_base.pkl","rb"))
        data=pickle.load(open(f"../working/data_for_analysis/HP_chpt2_{args.lm_name}_base_layer-1.pkl","rb"))
        words=data['text'][args.sequence_length:]
        MSEs=np.mean(data['MSE'][args.layer][12:17],axis=0)
    
    ranked_sentences = rank_sentences(words, MSEs)
    similar_sents = ranked_sentences[:args.n_sents]
    divergent_sents = ranked_sentences[-args.n_sents:]
    print("similar sentences:", similar_sents[:5])
    print("divergent sentences:", divergent_sents[-5:])
    
    ## Save data
    save_sents(divergent_sents, similar_sents, f"{args.save_path}formatted_sents_{args.lm_name}_layer{args.layer}_chpt{args.chapter}.pkl")


# ../working/interim_data/lm_embeddings/HP_chpt1_GPT2-xl_base.pkl

## TEST

In [ ]:
f"../working/interim_data/lm_embeddings/HP_chpt1_{args.lm_name}_base.pkl"

In [ ]:
for k in data.keys():
    try:
        print(k,data[k].shape)
    except:
        print(k,len(data[k]))

In [ ]:
args.sequence_length

In [ ]:
data2=pickle.load(open(f"../working/data_for_analysis/HP_chpt1_{args.lm_name}_base_layer-1.pkl","rb"))
for k in data2.keys():
    try:
        print(k,data2[k].shape)
    except:
        print(k,len(data2[k]))

## END OF TEST

# BEGINING THE ANALYSIS

### UseFull info
```python
word_num = meg_data.shape[0]
channel_num = meg_data.shape[1]
timepoint_num = meg_data.shape[2]


def single_topoplot(
    mat,
    cmap="RdBu_r",
    vmin=-0.1,
    vmax=0.1,
    figsize=(5,5),
    fontsize=16,
):
    """Creates single helmet plot for sensor-space MEG data (based on MNE visualization)

    Args:
        mat (2d numpy array): data to plot of size N sensors x M timepoints
        cmap (str, optional): colormap name. Defaults to 'RdBu_r'.
        vmin (float, optional): sets the colorbar min. Defaults to -0.1.
        vmax (float, optional): sets the colorbar max. Defaults to 0.1.
        figsize (tuple, optional): figure size. Defaults to (15,15).
        fontsize (int, optional): font size. Defaults to 16.

    Returns:
        figure handle
    """

def topoplot(
    mat,
    nrow=4,
    ncol=5,
    time_step=25,
    time_start=0,
    cmap="RdBu_r",
    vmin=-0.1,
    vmax=0.1,
    figsize=(15, 15),
    fontsize=16,
):
    """Creates helmet plots for sensor-space MEG data (based on MNE visualization)

    Args:
        mat (2d numpy array): data to plot of size N sensors x M timepoints
        nrow (int, optional): number of rows in plot. Defaults to 4.
        ncol (int, optional): number of columns in plot. Defaults to 5.
        time_step (int, optional): time window length. Defaults to 25.
        time_start (int, optional): what time to start plotting. Defaults to 0.
        cmap (str, optional): colormap name. Defaults to 'RdBu_r'.
        vmin (float, optional): sets the colorbar min. Defaults to -0.1.
        vmax (float, optional): sets the colorbar max. Defaults to 0.1.
        figsize (tuple, optional): figure size. Defaults to (15,15).
        fontsize (int, optional): font size. Defaults to 16.

    Returns:
        figure handle
    """
print("Ridge regression...")
for tt in range(timepoint_num):
    brain_responses = meg_data[:,:,tt]
    embeddings = all_embeddings[args.layer]
    brain_responses_pred = do_ridge_regression(embeddings, brain_responses)

    all_pred[0, tt] = brain_responses_pred

    for ch, (pred,actual) in enumerate(zip(brain_responses_pred.T, brain_responses.T)):	# for each channel, all words
        corr, pvalue = pearsonr(pred,actual,alternative="greater")
        all_corrs[0, tt, ch] = corr
        if pvalue < args.p_thresh:
            all_sig_channels[0, tt, ch] = 1

    sig_channels = all_sig_channels[0, tt]
    print(f"layer {args.layer}, {tt*25}-{(tt+1)*25}ms: {np.sum(sig_channels)}")

    for word, (pred,actual) in enumerate(zip(brain_responses_pred, brain_responses)):	# for each word, all sig channels
        all_MSE[0, tt, word] = np.linalg.norm(pred[sig_channels]-actual[sig_channels], ord=2) ** 2 / len(sig_channels)
        all_cos_sim[0, tt, word] = cosine_similarity(pred[sig_channels],actual[sig_channels])

```

sample rate = 1Khz

{tt*25}-{(tt+1)*25}

## PART 1 : FINDING BRAIN REGIONS 

In [ ]:
sig_channels.sum()

In [ ]:
print(all_MSE.shape)
print(brain_responses.shape)
print(sig_channels.shape)

In [ ]:
np.mean(all_MSE,axis=1).shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.mean(all_MSE,axis=1).T)

In [ ]:
def find_n_largest_values(arr,n=10,row=False):
    if row:
        # Get indices of the top-n values in each row (unsorted)
        indices_unsorted = np.argpartition(-arr, n, axis=1)[:, :n]
        # Gather the corresponding values
        row_indices = np.arange(arr.shape[0])[:, None]
        values_unsorted = arr[row_indices, indices_unsorted]
        
        # Optionally sort the top-n values within each row
        sorted_order = np.argsort(-values_unsorted, axis=1)
        values_sorted = np.take_along_axis(values_unsorted, sorted_order, axis=1)
        indices_sorted = np.take_along_axis(indices_unsorted, sorted_order, axis=1)
        
        print(f"Top-{n} values per row:\n", values_sorted)
        print(f"Indices of top-{n} values per row:\n", indices_sorted)
    else:
        # Get indices of the top-n values in each column (unsorted)
        indices_unsorted = np.argpartition(-arr, n, axis=0)[:n, :]
        # Gather the corresponding values
        col_indices = np.arange(arr.shape[1])
        values_unsorted = arr[indices_unsorted, col_indices]
        
        # Sort
        sorted_order = np.argsort(-values_unsorted, axis=0)
        values_sorted = np.take_along_axis(values_unsorted, sorted_order, axis=0)
        indices_sorted = np.take_along_axis(indices_unsorted, sorted_order, axis=0)
        
        print(f"Top-{n} values per column:\n", values_sorted)
        print(f"Indices of top-{n} values per column:\n", indices_sorted)
    return values_sorted,indices_sorted
find_n_largest_values(all_MSE[0,:,:],n=10,row=False)


In [ ]:
#Generate data for analysis HP
import os
import sys
import numpy as np
import string
import pickle
import argparse

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import zscore
import scipy.io as sio
from scipy.io import loadmat

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm

# from mat4py import loadmat

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../")
# from dataloader.dataloader import HP_dataset_denoised
# from utils import meg_sensor_plot
# from utils import utils
# from utils.ridge.ridge import bootstrap_ridge

np.set_printoptions(precision=3,suppress=True)

import warnings
warnings.filterwarnings("ignore")



home=os.path.expanduser("~")

parser = argparse.ArgumentParser()

parser.add_argument("--dataset", default="HP")
parser.add_argument("--base_data_path", default="../input/divergence-meg-data/")
parser.add_argument("--chapter", type=int, default=1)
parser.add_argument("--meg_offset", type=int, default=0)
parser.add_argument("--p_thresh", type=float, default=0.001)

parser.add_argument("--sequence_length", type=int, default=20)
parser.add_argument("--lm_name", default="GPT2-xl")
parser.add_argument("--model_info", default="base")
parser.add_argument("--layer", type=int, default=-1)


# --- FIX STARTS HERE ---
# Simulate command-line arguments by passing a list of strings

# Manually construct the list based on the defaults you want to use
args_list = [
    "--dataset", "HP",
    "--base_data_path", "../input/divergence-meg-data/", # Use the path variable
    "--chapter", "1",
    "--meg_offset", "0",
    "--p_thresh", "0.001", # Convert float to string

    "--sequence_length", "20",
    "--lm_name", "GPT2-xl",
    "--model_info", "base",
    "--layer", "-1"
]

# Pass the list of arguments to parse_args()
args = parser.parse_args(args_list) # <--- The fix is here
# --- FIX ENDS HERE ---


args.lm_embed_path = f"../working/interim_data/lm_embeddings/{args.dataset}_chpt{args.chapter}_{args.lm_name}_{args.model_info}.pkl"

print("Echo arguments:",args)

## Load MEG data
whole_data = HP_dataset_denoised(args)
meg_data=whole_data.all_megs[args.sequence_length:]	# (word, meg_channel, time)

## Load LM embeddings
data=pickle.load(open(args.lm_embed_path,"rb"))
all_embeddings=data["all_embeddings"]	# (layer, word, embedding_dim)

word_num = meg_data.shape[0]
channel_num = meg_data.shape[1]
timepoint_num = meg_data.shape[2]
layer_num = 1

## init arrays
all_corrs = np.zeros((layer_num, timepoint_num, channel_num))
all_sig_channels = np.zeros((layer_num, timepoint_num, channel_num), dtype=int)
all_pred = np.zeros((layer_num, timepoint_num, word_num, channel_num))
all_MSE = np.zeros((layer_num, timepoint_num, word_num))
all_cos_sim = np.zeros((layer_num, timepoint_num, word_num))

MSEs_for_channels = np.zeros((timepoint_num, channel_num))  # shape: (timepoints, channels)
cosine_sim_channels = np.zeros((timepoint_num, channel_num))  # shape: (timepoints, channels)
    for ch in range(channel_num):
        preds = brain_responses_pred[:, ch]    # shape: (words,) - predictions for one channel across all words
        actuals = brain_responses[:, ch]       # shape: (words,) - actuals for one channel across all words

        # MSE for this specific channel across all words
        mse = np.mean((preds - actuals) ** 2)
        MSEs_for_channels[tt, ch] = mse

        # Cosine Similarity for this specific channel across all words
        # Check if norms are non-zero to avoid division by zero in cosine_similarity if any array is all zeros
        if np.linalg.norm(preds) > 0 and np.linalg.norm(actuals) > 0:
            # Reshape to (1, -1) for sklearn.metrics.pairwise.cosine_similarity
            cos_sim = cosine_similarity(preds.reshape(1, -1), actuals.reshape(1, -1))[0, 0]
        else:
            cos_sim = 0.0 # Define behavior for all-zero vectors

        cosine_sim_channels[tt, ch] = cos_sim
for tt in range(timepoint_num):
    brain_responses = meg_data[:, :, tt]  # shape: (words, channels)
    embeddings = all_embeddings[args.layer]  # shape: (words, features)
    brain_responses_pred = do_ridge_regression(embeddings, brain_responses)  # shape: (words, channels)

    all_pred[0, tt] = brain_responses_pred

    # Pearson correlation per channel
    for ch in range(channel_num):
        pred = brain_responses_pred[:, ch]    # shape: (words,)
        actual = brain_responses[:, ch]       # shape: (words,)
        corr, pvalue = pearsonr(pred, actual, alternative="greater")
        all_corrs[0, tt, ch] = corr
        if pvalue < args.p_thresh:
            all_sig_channels[0, tt, ch] = 1

    sig_channels = all_sig_channels[0, tt] # sig_channels is (channel_num,) containing 0s or 1s
    print(f"layer {args.layer}, {tt*25}-{(tt+1)*25}ms: {np.sum(sig_channels)} significant channels")

    # Per-word metrics (on significant channels)
    for word in range(word_num): # Changed 'word' to 'word_idx' to avoid confusion if 'word' is a global
        pred = brain_responses_pred[word]     # (num_channels,) - all predictions for one word
        actual = brain_responses[word]        # (num_channels,) - all actuals for one word

        # Ensure there are significant channels for this calculation
        if np.sum(sig_channels) > 0:
            # np.linalg.norm(pred[sig_channels] - actual[sig_channels], ord=2)**2 calculates
            # the sum of squared differences for the selected significant channels for this word.
            # Dividing by len(sig_channels) makes it the Mean Squared Error (MSE).
            all_MSE[0, tt, word] = np.linalg.norm(pred[sig_channels == 1] - actual[sig_channels == 1], ord=2) ** 2 / np.sum(sig_channels == 1)
            # Reshape needed for sklearn.metrics.pairwise.cosine_similarity if inputs are 1D
            # Ensure not to pass empty arrays if no sig_channels exist for a word
            if pred[sig_channels == 1].size > 0:
                 all_cos_sim[0, tt, word] = cosine_similarity(pred[sig_channels == 1].reshape(1, -1), actual[sig_channels == 1].reshape(1, -1))[0, 0]
            else:
                 all_cos_sim[0, tt, word] = np.nan # Or 0.0, if desired for no sig channels
        else:
            all_MSE[0, tt, word] = np.nan # Or 0.0, if desired for no sig channels
            all_cos_sim[0, tt, word] = np.nan # Or 0.0, if desired for no sig channels


    # --- NEW: Compute per-channel MSE across words ---
    for ch in range(channel_num):
        preds = brain_responses_pred[:, ch]    # shape: (words,) - predictions for one channel across all words
        actuals = brain_responses[:, ch]       # shape: (words,) - actuals for one channel across all words

        # MSE for this specific channel across all words
        mse = np.mean((preds - actuals) ** 2)
        MSEs_for_channels[tt, ch] = mse

        # Cosine Similarity for this specific channel across all words
        # Check if norms are non-zero to avoid division by zero in cosine_similarity if any array is all zeros
        if np.linalg.norm(preds) > 0 and np.linalg.norm(actuals) > 0:
            # Reshape to (1, -1) for sklearn.metrics.pairwise.cosine_similarity
            cos_sim = cosine_similarity(preds.reshape(1, -1), actuals.reshape(1, -1))[0, 0]
        else:
            cos_sim = 0.0 # Define behavior for all-zero vectors

        cosine_sim_channels[tt, ch] = cos_sim


In [ ]:
args

In [ ]:
single_topoplot(np.mean(meg_data[0,:,:],axis=1))

In [ ]:
dir()

In [ ]:
len(whole_data.all_texts)

# END OF WORKING CODE

# Generate data for analysis Moth

In [ ]:
import os
import sys
import numpy as np
import string
import pickle
import argparse

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import zscore
import scipy.io as sio
from scipy.io import loadmat

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

import statsmodels.api as sm

# from mat4py import loadmat

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../")
from dataloader.dataloader import HP_dataset_denoised
from utils import meg_sensor_plot
from utils import utils
from utils.ridge.ridge import bootstrap_ridge

np.set_printoptions(precision=3,suppress=True)

import warnings
warnings.filterwarnings("ignore")

def construct_stimuli_and_brain_responses(MEG, word_embeddings, word_onset_point, story_dict, args, layer = -1, PCA_n_components = 50, n_delays = 40, do_zscore = True):
	"""construct stimuli and brain responses for regression"""
	
	# construct brain responses
	brain_responses = []
	for story in story_dict:
		avg_MEG = []
		for trial in story_dict[story]:
			if do_zscore:
				avg_MEG.append(zscore(MEG[trial]))
			else:
				avg_MEG.append(MEG[trial])
		brain_responses.append(np.stack(avg_MEG).mean(axis=0))
	brain_responses = np.concatenate(brain_responses, axis = 0)	# (word, channel)

	stimuli = np.zeros((brain_responses.shape[0], word_embeddings.shape[2]))	# (word, embedding_dim)

	# construct stimuli
	offset = 0
	idx = 0
	for i, story in enumerate(story_dict):
		trial = story_dict[story][0]
		for time in word_onset_point[trial][args.sequence_length:]:
			stimuli[time+offset] = word_embeddings[args.layer][idx]
			idx += 1
		offset += MEG[trial].shape[0]

	# reduce dimensionality of stimuli
	pca = PCA(n_components=PCA_n_components).fit(stimuli)
	stimuli = pca.transform(stimuli)	# (word, PCA_n_components)
	
	# add delays
	stimuli = utils.delay_mat(stimuli, range(n_delays))	# (word, PCA_n_components*n_delays)

	return stimuli, brain_responses

if __name__ == '__main__':
	home=os.path.expanduser("~")

	parser = argparse.ArgumentParser()
	
	parser.add_argument("--dataset", default="Moth")
	parser.add_argument("--base_data_path", default=f"{home}/Desktop/MEG_divergence/Moth_data/")
	parser.add_argument("--p_thresh", type=int, default=0.001)

	parser.add_argument("--sequence_length", type=int, default=20)
	parser.add_argument("--lm_embed_path", default=f"{home}/Desktop/MEG_divergence/interim_data/lm_embeddings/Moth_GPT2-xl_base.pkl")
	parser.add_argument("--lm_name", default="GPT2-xl")
	parser.add_argument("--model_info", default="base")
	parser.add_argument("--layer", type = int, default=-1)


	args = parser.parse_args()

	print("Echo arguments:",args)
	
	## Load data

	MEG = pickle.load(open("../Moth_data/Moth_MEG.pkl","rb"))
	word_onset_point = pickle.load(open("../Moth_data/Moth_word_onset_point.pkl","rb"))
	word_embeddings = pickle.load(open(args.lm_embed_path,"rb"))["all_embeddings"]
	story_dict = pickle.load(open("../Moth_data/Moth_run_info_test_story_dict.pkl","rb"))

	# Construct matrix of features

	all_embeddings, meg_data = construct_stimuli_and_brain_responses(MEG, word_embeddings, word_onset_point, story_dict, args)
	all_embeddings = np.expand_dims(all_embeddings, axis=0)
	meg_data = np.expand_dims(meg_data, axis=2)

	print("embedding shape:", all_embeddings.shape)
	print("meg_data shape:", meg_data.shape)

	word_num = meg_data.shape[0]
	channel_num = meg_data.shape[1]
	timepoint_num = 1
	layer_num = 1

	## init arrays
	all_corrs = np.zeros((layer_num, timepoint_num, channel_num))
	all_sig_channels = np.zeros((layer_num, timepoint_num, channel_num), dtype=int)
	all_pred = np.zeros((layer_num, timepoint_num, word_num, channel_num))
	all_MSE = np.zeros((layer_num, timepoint_num, word_num))
	all_cos_sim = np.zeros((layer_num, timepoint_num, word_num))

	## Ridge regression
	print("Ridge regression...")
	for layer in range(layer_num):
		for time in range(timepoint_num):
			brain_responses = meg_data[:,:,time]
			embeddings = all_embeddings[layer]
			brain_responses_pred = utils.do_ridge_regression(embeddings, brain_responses)

			all_pred[layer, time] = brain_responses_pred

			for ch, (pred,actual) in enumerate(zip(brain_responses_pred.T, brain_responses.T)):	# for each channel, all words
				corr, pvalue = pearsonr(pred,actual,alternative="greater")
				all_corrs[layer, time, ch] = corr
				if pvalue < args.p_thresh:
					all_sig_channels[layer, time, ch] = 1

			sig_channels = all_sig_channels[layer, time]
			print(f"layer {args.layer}: {np.sum(sig_channels)}")

			for word, (pred,actual) in enumerate(zip(brain_responses_pred, brain_responses)):	# for each word, all sig channels
				all_MSE[layer, time, word] = np.linalg.norm(pred[sig_channels]-actual[sig_channels], ord=2) ** 2 / len(sig_channels)
				all_cos_sim[layer, time, word] = utils.cosine_similarity(pred[sig_channels],actual[sig_channels])


	pickle.dump(
		dict(
		embedding=all_embeddings,
		actual=meg_data,
		pred=all_pred,
		corr=all_corrs,
		MSE=all_MSE,
		cos=all_cos_sim,
		sig_channels=all_sig_channels,
		),
		open(f"../interim_data/data_for_analysis/{args.dataset}_{args.lm_name}_{args.model_info}.pkl","wb"))


# Generate LM Embedding Moth

In [ ]:
import os
import sys
import numpy as np
import string
import pickle
import argparse

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import chi2
from scipy.stats import zscore
import scipy.io as sio
from scipy.io import loadmat

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.api as sm

# from mat4py import loadmat

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split

sys.path.append("../")
from dataloader.dataloader import HP_dataset_denoised
from utils import meg_sensor_plot
from utils import utils
from utils.ridge.ridge import bootstrap_ridge

np.set_printoptions(precision=3,suppress=True)

import warnings
warnings.filterwarnings("ignore")


if __name__ == '__main__':
	home=os.path.expanduser("~")

	parser = argparse.ArgumentParser()
	
	parser.add_argument("--dataset", default="Moth")
	parser.add_argument("--base_data_path", default=f"{home}/Desktop/MEG_divergence/Moth_data/")
	# parser.add_argument("--chapter", type=int, default=1)
	# parser.add_argument("--meg_offset", type=int, default=0)

	parser.add_argument("--batch_size", type=int, default=1)
	parser.add_argument("--sequence_length", type=int, default=20)
	parser.add_argument("--lm_name", default="GPT2-xl")
	parser.add_argument("--model_info", default="base")
	parser.add_argument("--lm_path")
	
	args = parser.parse_args()
	args.no_cuda=not torch.cuda.is_available()

	print("Echo arguments:",args)

	## Load dataset
	Moth_words = pickle.load(open("../Moth_data/Moth_words.pkl","rb"))
	Moth_run_info_test_story_dict = pickle.load(open("../Moth_data/Moth_run_info_test_story_dict.pkl","rb"))

	all_sents = []
	for story in Moth_run_info_test_story_dict:
		st = Moth_run_info_test_story_dict[story][0]
		for i in range(args.sequence_length, len(Moth_words[st])):
			all_sents.append(" ".join(Moth_words[st][i-args.sequence_length:i]).lower())
	print("Number of sentences:", len(all_sents))

	## Load LM
	if args.model_info=="base":	# if loading base model
		tokenizer,model=utils.load_tokenizer_and_model_from_transformers(args.lm_name)
	else:	# if loading finetuned model
		from transformers import AutoTokenizer, AutoModelForCausalLM
		if args.lm_name == "Llama-2":
			tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", truncation_side="left", padding_side="left")
		elif args.lm_name == "GPT2-xl":
			tokenizer = AutoTokenizer.from_pretrained("gpt2-xl", truncation_side="left", padding_side="left")
		else:
			raise ValueError("Not a valid lm_name.")
		model = AutoModelForCausalLM.from_pretrained(args.lm_path, output_hidden_states=True)
		
	tokenizer.pad_token = tokenizer.bos_token

	if not args.no_cuda:
		model=model.to("cuda:0")
	model.eval();

	## Get LM embeddings

	all_embeddings= []

	for i, text in enumerate(all_sents):
		encodings = tokenizer(text, return_tensors="pt", padding=True)['input_ids']
		# encodings = tokenizer(batch_text, return_tensors="pt", truncation=True, max_length=args.sequence_length)['input_ids']
		if not args.no_cuda:
			encodings=encodings.to("cuda:0")

		last_word = text.split(" ")[-1]
		last_word_pos = utils.find_indices_of_last_word_in_batch(tokenizer,[last_word],encodings)
		word_start, word_end = last_word_pos[0]

		with torch.no_grad():
			output=model(encodings)
			embeddings=np.vstack([torch.mean(embed[0, word_start:word_end+1],dim=0).detach().cpu().numpy()  for embed in output['hidden_states']])
			all_embeddings.append(embeddings)

		if i % 100 == 0:
			print(f"Finished {i} out of {len(all_sents)}")

	all_embeddings=np.stack(all_embeddings,axis=1)	# (layer, word, embedding_dim)

	dumped_data=dict(
			all_embeddings=all_embeddings,
		)

	pickle.dump(dumped_data,
		open(f"{home}/Desktop/MEG_divergence/interim_data/lm_embeddings/{args.dataset}_{args.lm_name}_{args.model_info}.pkl","wb"))


# Fine Tune with Commonsense